In [1]:
import tensorflow as tf

import numpy as np

import matplotlib.pyplot as plt

%matplotlib notebook

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
x = tf.placeholder(tf.float32, shape=[None, 784],name="inputs")
y_ = tf.placeholder(tf.float32, shape=[None, 10],name="targets")

## Helper Functions

In [4]:
# define some helper functions to abstract away the tensorflow interface
## and make the network-building code cleaner

def weight_variable(shape,name):
    initial = tf.truncated_normal(shape, stddev=0.1, name=name)
    return tf.Variable(initial)

def bias_variable(shape,name):
    initial = tf.constant(0.1, shape=shape, name=name)
    return tf.Variable(initial)

def conv2d(x, W,name):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME', name=name)

def max_pool_2x2(x,name):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME', name=name)

## Convolution 1

In [5]:
x_image = tf.reshape(x, [-1, 28, 28, 1],name="reshaped_input")

W_conv1 = weight_variable([5, 5, 1, 32],'weights_convolution_1')
b_conv1 = bias_variable([32],'biases_convolution_1')

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1,'output_convolution_1') 
                             + b_conv1,
                    name="activations_convolution_1")
h_pool1 = max_pool_2x2(h_conv1,"activations_pool_1")

## Convolution 2

In [6]:
W_conv2 = weight_variable([5, 5, 32, 64],'weights_convolution_2')
b_conv2 = bias_variable([64], 'biases_convolution_2')

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2,'output_convolution_2') 
                             + b_conv2,
                    name="activations_convolution_2")

h_pool2 = max_pool_2x2(h_conv2,"activations_pool_2")

## Fully-Connected 1

In [7]:
W_fc1 = weight_variable([7 * 7 * 64, 1024],'weights_fullyconnected_1')
b_fc1 = bias_variable([1024],'biases_fullyconnected_1')

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64],
                         name="reshapedactivations_hidden_2")
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1,
                            name="output_fullyconnected_1") 
                   + b_fc1,
                  name="activations_fullyconnected_1")

## DropOut

In [8]:
keep_prob = tf.placeholder(tf.float32,name="dropout_keep_probability")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob,
                          name="droppedactivations_fullyconnected_1")

In [9]:
W_fc2 = weight_variable([1024, 10],"weights_fullyconnected_2")
b_fc2 = bias_variable([10], "biases_fullyconnected_2")

y_conv = tf.add(tf.matmul(h_fc1_drop, W_fc2,
                  name="outputs_fullyconnected_2"),
                b_fc2,name="activations_fullyconnected_2")

In [ ]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv,
                                           name="crossentropy_cost"),
                name="mean_crossentropy_cost")

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv, 1,
                                       name="MAPestimate_index"), 
                              tf.argmax(y_, 1,
                                       name="label_index"),
                             name="MAP_iscorrect")

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32),
                         name="fraction_correct")

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        batch = mnist.train.next_batch(50)
        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict={
                                x: batch[0], y_: batch[1], keep_prob: 1.0})
            print('step %d, training accuracy %g' % (i, train_accuracy))
                
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    print('test accuracy %g' % accuracy.eval(feed_dict={
                x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.06


In [17]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    weights = W_conv1.eval()

In [ ]:
numFilters = weights.shape[-1]
randomWeight = np.random.choice(numFilters)